# 线性回归的简洁实现

In [1]:
import numpy as np
import torch
from torch.utils import data
import d2l.torch as d2l
import matplotlib.pyplot as plt
from torch import nn

随机生成训练数据集

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = torch.tensor([4.2])
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

构造一个 $torch$ 的数据迭代器

`TensorDataset` 可以理解为将数据转换为一个可以供后面的 $DataLoader$ 使用的参数，$*$ 有序列解包的作用, 

In [3]:
def load_array(data_arrays, bathch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, bathch_size, shuffle=is_train)

$batch\_size$ 太大其实是一件不那么好的事情，$batch\_size$ 很小的情况下，其实是一件好事，因为小的话鲁棒性更高，噪音更大，噪音对神经网络是一件好事情，多一点噪音会让他走偏一点，让泛化性更高

In [4]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

$Sequential$ 可以理解成为一个 $list$ $of$ $layer$, Linear 是线性层 $y = xA^T + b$

In [5]:
net = nn.Sequential(nn.Linear(2,1))

初始化模型参数

$net[0]$ 是指容器里面的第一层，也就是我们的 $linear$

In [6]:
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是 $MSELoss$ 类，也就是 $L_2$ 范数

$SGD$ 是 $Stochastic$ $Gradient$ $Descent$ 

In [7]:
LearningRate = 0.03
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(),lr=LearningRate)

训练过程

In [8]:
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        trainer.step() # 更新
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000189
epoch 2, loss 0.000098
epoch 3, loss 0.000098
